### Access the data

In [263]:
import pandas as pd

# Load the dataset as pd
df = pd.read_csv("./dataset/fliptop_videos.csv")
df.shape


(1748, 7)

### Understand the Structure

In [264]:
# df.head()
# df.info()
df.describe()
# df.dtypes

,view_count,like_count,comment_count
count,1.748000e+03,1748.00000,1748.000000
mean,1.542244e+06,11369.23341,1952.402174
std,3.956757e+06,29366.72784,3907.400216
min,5.375000e+03,0.00000,0.000000
25%,1.203922e+05,732.25000,241.000000
50%,4.089450e+05,2633.50000,754.000000
75%,1.274661e+06,10994.75000,2119.750000
max,5.918583e+07,549488.00000,77144.000000


### Assess Data Quality

In [265]:
# check for null records
df.isnull().sum()
df[df["description"].isnull()]

# check for duplicate observations
df.duplicated()
df[df.duplicated()]

,video_id,title,description,published_at,view_count,like_count,comment_count


### Filter out non-battle titles/videos

In [266]:
# filter titles with "vs" substring
df = df[df["title"].str.contains("vs", case=True)]
df = df[df["title"].str.startswith("FlipTop - ")]
df = df.drop(["video_id", "description"], axis=1) # remove video_id column
df.head(3)

,title,published_at,view_count,like_count,comment_count
4,FlipTop - Negho Gy vs Hespero,2025-06-25T12:44:16Z,322865,4561,756
6,FlipTop - Ruffian vs JDee,2025-06-21T12:23:04Z,607827,12438,2161
10,FlipTop - Katana vs 3rdy @ Isabuhay 2025,2025-06-14T12:14:27Z,1035763,15851,3838


### Removing also Royal Rumbles

In [267]:
df = df[~df["title"].str.contains("royal rumble", case=False)]
df = df[~df["title"].str.contains("Shernan vs M Zhayt vs Pistolero vs Lhipkram vs Romano", case=False)]
df = df[~df["title"].str.contains("5 on 5 Battle", case=False)]

### Transform "published_at" and add Date Fields

In [268]:
df["published_at"] = pd.to_datetime(df["published_at"])
df["date"] = df["published_at"].dt.date
df["year"] = df["published_at"].dt.year
df["month"] = df["published_at"].dt.month
df["week"] = df["published_at"].dt.isocalendar().week

df = df.drop("published_at", axis=1)
df.head(3)

,title,view_count,like_count,comment_count,date,year,month,week
4,FlipTop - Negho Gy vs Hespero,322865,4561,756,2025-06-25,2025,6,26
6,FlipTop - Ruffian vs JDee,607827,12438,2161,2025-06-21,2025,6,25
10,FlipTop - Katana vs 3rdy @ Isabuhay 2025,1035763,15851,3838,2025-06-14,2025,6,24


### Extract Emcee Name in the Title column

In [269]:
# using regular expression for extracting emcee names

df['emcee_1'] = df['title'].str.extract(r'FlipTop - (.*?) vs ')
df['emcee_2'] = df['title'].str.extract(r' vs (.*?)(?: @| -| \||\*| pt\.| \(|$)')

df.sample(5)

# df.loc[[1400,620,1234,1736]]

,title,view_count,like_count,comment_count,date,year,month,week,emcee_1,emcee_2
1373,FlipTop - Rudic vs Hizuka,106895,181,91,2014-01-16,2014,1,3,Rudic,Hizuka
802,FlipTop - Climax vs Gustav,99564,832,271,2019-06-21,2019,6,25,Climax,Gustav
1165,FlipTop - EJ Power vs El Quiel,409815,1729,374,2016-04-13,2016,4,15,EJ Power,El Quiel
1348,FlipTop - Thike vs Ejo,128361,289,124,2014-04-06,2014,4,14,Thike,Ejo
603,FlipTop - Batas vs J-King @ Isabuhay 2020 | Qu...,1322170,24391,2705,2020-09-05,2020,9,36,Batas,J-King


### Duplicating the data
##### Each emcee is considered contributor to the battle's performance metrics (views, likes, comments, etc). 

In [270]:
df1 = df[["emcee_1", "view_count", "like_count", "comment_count", "date", "year", "month", "week"]].rename(columns={"emcee_1": "emcee_name"})
df2 = df[["emcee_2", "view_count", "like_count", "comment_count", "date", "year", "month", "week"]].rename(columns={"emcee_2": "emcee_name"})
df_full = pd.concat([df1, df2], ignore_index=True)


In [271]:
# save data for manual validation
df_full.to_csv("dataset/test_file.csv", index=False, encoding="utf-8")

### Exploratory Data Analysis